In [1]:
import time
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt

In [2]:
ORIGINAL_PB_FILE = '/tf_api/workspace/training/output/output_inference_graph_v3.pb/frozen_inference_graph.pb'
OPTIMIZED_FP16_GRAPH = '/experiments/models/converted/ssd_resnet50_v1_fpn_fp16.pb'

In [3]:
with tf.gfile.GFile(ORIGINAL_PB_FILE, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [4]:
# Configure TF session
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

In [5]:
SEGMENT_SIZE = 3
IS_DYNAMIC = False
WORKSPACE_SIZE = 2*10**9
BATCH_SIZE = 1  
NUM_ENGINES = 1
BATCH_SIZES = []

In [6]:
with tf.Session(config=tf_config): 
    start = time.perf_counter()
    trt_graph = trt.create_inference_graph(
        input_graph_def=graph_def,
        outputs = ['detection_boxes:0','detection_scores:0','detection_classes:0','num_detections:0'],
        max_batch_size=BATCH_SIZE,
        max_workspace_size_bytes=WORKSPACE_SIZE,
        is_dynamic_op=False,
        precision_mode='FP16',
        maximum_cached_engines=NUM_ENGINES,
        #cached_engine_batch_sizes=BATCH_SIZES, 
        #cached_engine_batches=BATCH_SIZES,
        minimum_segment_size=SEGMENT_SIZE
    )
    print(time.perf_counter() - start)

start = time.perf_counter()
with open(OPTIMIZED_FP16_GRAPH, 'wb') as f:
    f.write(trt_graph.SerializeToString())
print(time.perf_counter() - start)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0
12.279801943999985
0.13442467699996996
